In [1]:
from langchain_openai import OpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import JsonOutputToolsParser
import os
import json
from dotenv import load_dotenv

/Users/chetanvallish/Conscious/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
load_dotenv()

search_tool = TavilySearchResults(api_key=os.getenv("TAVILY_API_KEY"), search_type="news", num_results=10)
tools = [search_tool]

# Initialize model
llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    top_p=1.0,
    max_tokens=500,
)

In [31]:
company_name = "Nestle"
query = f"List the main atrocities committed by {company_name} against climate and human rights"
articles = search_tool.invoke(query)
context = "\n\n".join([article["title"] + "\n"+ article["content"] for article in articles])
prompt = f"""Context:
{context}

Question: {query}
Try and get atleast 5 main points about the atrocities committed by {company_name} against climate and human rights.
in the answer, include one link as the main source at the end of each point.
Also include the Date of when the accusation of the atrocity has been made. 
Return the answer ONLY in JSON format as below:
{{
    'points': [
        {{
            'point': 'Summary of point 1',
            'date': 'Date of accusation',
            'source': 'Link to main source',
        }},
        {{
            'point': 'Summary of point 2',
            'date': 'Date of accusation',
            'source': 'Link to main source
        }},
        {{
            'point': 'Summary of Point 3',
            'date': 'Date of accusation',
            'source': 'Link to main source'
        }},
        {{
            'point': 'Summary of Point 4',
            'date': 'Date of accusation',
            'source': 'Link to main source'
        }},
        {{
            'point': 'Summary of Point 5',
            'date': 'Date of accusation',
            'source': 'L'
        }}
    ]
}}
"""
response = llm.invoke(prompt)

In [32]:
raw_content = response.content if hasattr(response, 'content') else response

In [33]:
print(raw_content) # Parse the response using the JSON output parser

```json
{
    "points": [
        {
            "point": "Nestlé has been linked to child labor and human rights violations in its cocoa supply chain, with thousands of children working under forced conditions.",
            "date": "2021",
            "source": "https://www.greendigest.com/nestle-human-rights-violations"
        },
        {
            "point": "The company has been involved in extensive union-busting activities in Colombia, creating a climate of fear among workers.",
            "date": "2021",
            "source": "https://www.greendigest.com/nestle-union-busting-colombia"
        },
        {
            "point": "Nestlé has faced accusations of using HFC refrigerants and failing to report its carbon emissions, contributing to climate change.",
            "date": "2020",
            "source": "https://www.greendigest.com/nestle-climate-change"
        },
        {
            "point": "The company has been criticized for its plastic pollution, being one of the l

In [34]:
from langchain.output_parsers.json import SimpleJsonOutputParser
parser = SimpleJsonOutputParser()
response = parser.parse(raw_content)

In [35]:
for points in response['points']:
    print(f"Point: {points['point']}, Source: {points['source']}, Date: {points['date']}")  # Print each point and its source

Point: Nestlé has been linked to child labor and human rights violations in its cocoa supply chain, with thousands of children working under forced conditions., Source: https://www.greendigest.com/nestle-human-rights-violations, Date: 2021
Point: The company has been involved in extensive union-busting activities in Colombia, creating a climate of fear among workers., Source: https://www.greendigest.com/nestle-union-busting-colombia, Date: 2021
Point: Nestlé has faced accusations of using HFC refrigerants and failing to report its carbon emissions, contributing to climate change., Source: https://www.greendigest.com/nestle-climate-change, Date: 2020
Point: The company has been criticized for its plastic pollution, being one of the largest contributors to ocean pollution through its single-use plastic packaging., Source: https://www.utopia.com/nestle-plastic-pollution, Date: 2021
Point: Nestlé has been accused of extracting millions of gallons of water from California creeks, leading to